In [ ]:
# https://drive.google.com/file/d/1L5b3wN8buM5avRSPJaGHDL_nCsI1avfR/view?usp=sharing

!gdown 1L5b3wN8buM5avRSPJaGHDL_nCsI1avfR

Downloading...
From: https://drive.google.com/uc?id=1L5b3wN8buM5avRSPJaGHDL_nCsI1avfR
To: /content/VietNameseSQaD.zip
100% 13.8M/13.8M [00:00<00:00, 155MB/s]


In [ ]:
!unzip '/content/VietNameseSQaD.zip' -d path_to_directory

Archive:  /content/VietNameseSQaD.zip
  inflating: path_to_directory/dev-v2.0-translated.json  
  inflating: path_to_directory/train-v2.0-translated.json  


In [ ]:
!pip install --quiet pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 28.6 MB/s eta 0:00:00


## Tập dữ liệu
The Stanford Question Answering Dataset, translated to Vietnamese\
Download here: https://drive.google.com/file/d/1L5b3wN8buM5avRSPJaGHDL_nCsI1avfR/view?usp=sharing

Cấu trúc:
- Context
- Câu hỏi
- Câu trả lời

Đánh giá dữ liệu thiếu nhiều câu trả lời, chưa thực sự chất lượng, câu trả lời quá context câu hỏi khá ngắn.\
Dễ thực hiện huấn luyện nhỏ


In [ ]:
from typing import List, Dict
import tqdm.notebook as tq
from tqdm.notebook import tqdm
import json
import pandas as pd
import numpy as np

import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
    )

In [ ]:
import json
#read dataset from json file
with open('/content/path_to_directory/train-v2.0-translated.json', 'r') as f:
    data = json.load(f)

# data

In [ ]:
for i in range(3):

  print('Context: ',data[i][0]) # question
  print('Question: ',data[i][1]) # context
  print('Answer: ',data[i][2]) # answer
  print()

Context:  Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên ở Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny's Child. Được quản lý bởi cha cô, Mathew Knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 "Crazy in Love" và "Baby Boy".
Question:  Beyonce bắt đầu nổi tiếng từ khi nào?
Answer:  Vào cuối những năm 1990

Context:  Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu 

In [ ]:
import pandas as  pd

df = pd.DataFrame(data, columns=['context', 'question', 'answer_text'])
df

,context,question,answer_text
0,Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s...,Beyonce bắt đầu nổi tiếng từ khi nào?,Vào cuối những năm 1990
1,Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s...,Beyonce đã cạnh tranh trong lĩnh vực nào khi c...,Hát và nhảy múa
2,Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s...,Beyonce rời Destiny's Child và trở thành ca sĩ...,2003
3,Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s...,Beyonce lớn lên ở thành phố và tiểu bang nào?,Houston Khách sạn
4,Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s...,Beyonce nổi tiếng trong thập kỷ nào?,Cuối những năm 1990
...,...,...,...
130314,"Thuật ngữ ""vật chất"" được sử dụng trong vật lý...",Vật lý đã đồng ý rộng rãi về định nghĩa của nh...,
130315,"Thuật ngữ ""vật chất"" được sử dụng trong vật lý...",Ai đã đặt ra thuật ngữ vấn đề Partonic?,
130316,"Thuật ngữ ""vật chất"" được sử dụng trong vật lý...",Một cái tên khác cho phản vật chất là gì?,
130317,"Thuật ngữ ""vật chất"" được sử dụng trong vật lý...",Vật chất thường không cần phải được sử dụng kế...,


In [ ]:
import pandas as pd
import numpy as np

df_real = df.copy()

df_real.replace("", np.nan, inplace=True)

df_real = df_real.dropna(subset=['context', 'question', 'answer_text'])

df_real


,context,question,answer_text
0,Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s...,Beyonce bắt đầu nổi tiếng từ khi nào?,Vào cuối những năm 1990
1,Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s...,Beyonce đã cạnh tranh trong lĩnh vực nào khi c...,Hát và nhảy múa
2,Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s...,Beyonce rời Destiny's Child và trở thành ca sĩ...,2003
3,Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s...,Beyonce lớn lên ở thành phố và tiểu bang nào?,Houston Khách sạn
4,Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s...,Beyonce nổi tiếng trong thập kỷ nào?,Cuối những năm 1990
...,...,...,...
130046,"Thành phố Thủ đô Kathmandu (KMC), nhằm thúc đẩ...",Kathmandu lần đầu tiên thiết lập một mối quan ...,Oregon
130047,"Thành phố Thủ đô Kathmandu (KMC), nhằm thúc đẩ...",Yangon trước đây được biết đến như thế nào?,Rangoon
130048,"Thành phố Thủ đô Kathmandu (KMC), nhằm thúc đẩ...",Với những gì thành phố Belorussian Kathmandu c...,Minsk
130049,"Thành phố Thủ đô Kathmandu (KMC), nhằm thúc đẩ...",Kathmandu đã tạo ra mối quan hệ quốc tế ban đầ...,1975


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split



sampled_df = df_real
print("df sampled size: ", sampled_df.shape)

train_df, temp_df = train_test_split(sampled_df, test_size=0.4, random_state=42)
test_df, dev_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print("Train set size: ", train_df.shape)
print("Test set size: ", test_df.shape)
print("Dev set size: ", dev_df.shape)



df sampled size:  (86820, 3)
Train set size:  (52092, 3)
Test set size:  (17364, 3)
Dev set size:  (17364, 3)


In [ ]:
dev_df.iloc[0]['context']

'Các mô tả lưu trữ về các tài sản cố định của chính phủ liên bang đang được NARA quản lý được lưu trữ trong Danh mục nghiên cứu lưu trữ (ARC). Các mô tả lưu trữ bao gồm thông tin về các tài sản giấy truyền thống, hồ sơ điện tử và hiện vật. Tính đến tháng 12 năm 2012, danh mục bao gồm khoảng 10 tỷ hồ sơ dữ liệu logic mô tả 527.000 hiện vật và bao gồm 81% hồ sơ của NARA. Ngoài ra còn có 922.000 bản sao kỹ thuật số của các tài liệu đã được số hoá.'

# MASK

In [ ]:
SEP_TOKEN = '<sep>'
MASKING_CHANCE = 0.3 #30% chance to replace the answer with '[MASK]'

In [ ]:
class QGDataset(Dataset):

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: T5Tokenizer,
        source_max_token_len: int,
        target_max_token_len: int
        ):

        self.tokenizer = tokenizer
        self.data = data
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        if np.random.rand() > MASKING_CHANCE:
            answer = data_row['answer_text']
        else:
            answer = '[MASK]'

        source_encoding = tokenizer(
            '{} {} {}'.format(answer, SEP_TOKEN, data_row['context']),
            max_length= self.source_max_token_len,
            padding='max_length',
            truncation= True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
            )

        target_encoding = tokenizer(
            '{} {} {}'.format(data_row['answer_text'], SEP_TOKEN, data_row['question']),
            max_length=self.target_max_token_len,
            padding='max_length',
            truncation = True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
            )

        labels = target_encoding['input_ids']
        labels[labels == 0] = -100

        return dict(
            answer_text = data_row['answer_text'],
            context = data_row['context'],
            question = data_row['question'],
            input_ids = source_encoding['input_ids'].flatten(),
            attention_mask = source_encoding['attention_mask'].flatten(),
            labels=labels.flatten()
            )

In [ ]:
class QGDataModule(pl.LightningDataModule):

    def __init__(
        self,
        train_df: pd.DataFrame,
        val_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size,
        source_max_token_len: int,
        target_max_token_len: int
        ):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def setup(self,stage=None):
        self.train_dataset = QGDataset(self.train_df, self.tokenizer, self.source_max_token_len, self.target_max_token_len)
        self.val_dataset = QGDataset(self.val_df, self.tokenizer, self.source_max_token_len, self.target_max_token_len)
        self.test_dataset = QGDataset(self.test_df, self.tokenizer, self.source_max_token_len, self.target_max_token_len)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size = self.batch_size, shuffle=True, num_workers = 2)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=1, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=1, num_workers=2)

# Mô hình Vit5-base
...

In [ ]:
MODEL_NAME = 'VietAI/vit5-base'
SOURCE_MAX_TOKEN_LEN = 300
TARGET_MAX_TOKEN_LEN = 80

N_EPOCHS = 1
BATCH_SIZE = 16
LEARNING_RATE = 0.001

In [ ]:
DF_TAKE_PERCENTAGE = 1

TAKE_TRAIN = int(len(train_df) * DF_TAKE_PERCENTAGE)
TAKE_DEV = int(len(dev_df) * DF_TAKE_PERCENTAGE)
TAKE_TEST = int(len(test_df) * DF_TAKE_PERCENTAGE)

print('Taking', DF_TAKE_PERCENTAGE * 100, '%')
print(TAKE_TRAIN, 'of', len(train_df))
print(TAKE_DEV, 'of', len(dev_df))
print(TAKE_TEST, 'of', len(test_df))

Taking 100 %
52092 of 52092
17364 of 17364
17364 of 17364


In [ ]:
print(train_df[:TAKE_TRAIN].shape, dev_df[:TAKE_DEV].shape, test_df[:TAKE_TEST].shape)

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
print('tokenizer len before: ', len(tokenizer))
tokenizer.add_tokens(SEP_TOKEN)
print('tokenizer len after: ', len(tokenizer))
TOKENIZER_LEN = len(tokenizer)

data_module = QGDataModule(train_df[:TAKE_TRAIN], dev_df[:TAKE_DEV], test_df[:TAKE_TEST], tokenizer, BATCH_SIZE, SOURCE_MAX_TOKEN_LEN, TARGET_MAX_TOKEN_LEN)
data_module.setup()

(52092, 3) (17364, 3) (17364, 3)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

tokenizer len before:  36096
tokenizer len after:  36097


In [ ]:
class QGModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        self.model.resize_token_embeddings(TOKENIZER_LEN) #resizing after adding new tokens to the tokenizer

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=LEARNING_RATE)

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
        dirpath='checkpoints',
        filename='best-checkpoint',
        save_top_k=-1,
        verbose=True,
        monitor='val_loss',
        mode='min'
    )

trainer = pl.Trainer(
        callbacks = checkpoint_callback,
        max_epochs=N_EPOCHS,
        devices=1,
    )


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
model = QGModel()
# model = QGModel.load_from_checkpoint('checkpoints/best-checkpoint-v42.ckpt')

trainer.fit(model, data_module)

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params | Mode
------------------------------------------------------------
0 | model | T5ForConditionalGeneration | 225 M  | eval
------------------------------------------------------------
225 M     Trainable params
0         Non-trainable params
225 M     Total params
903.807   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 3256: 'val_loss' reached 1.99868 (best 1.99868), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
checkpoint_path = '/content/checkpoints/best-checkpoint.ckpt'

best_model = QGModel.load_from_checkpoint(checkpoint_path)
best_model.freeze()
best_model.eval()

print()

In [ ]:
def generate(qgmodel: QGModel, answer: str, context: str) -> str:
    source_encoding = tokenizer(
        '{} {} {}'.format(answer, SEP_TOKEN, context),
        max_length=SOURCE_MAX_TOKEN_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
     # Move inputs to the same device as the model
    device = next(qgmodel.parameters()).device
    source_encoding['input_ids'] = source_encoding['input_ids'].to(device)
    source_encoding['attention_mask'] = source_encoding['attention_mask'].to(device)


    generated_ids = qgmodel.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=1,
        max_length=TARGET_MAX_TOKEN_LEN,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=True
    )

    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }

    return ''.join(preds)

In [ ]:
def show_result(generated: str, answer: str, context:str, original_question: str = ''):
    print('Generated: ', generated)
    if original_question:
        print('Original : ', original_question)

    print()
    print('Answer: ', answer)
    print('Conext: ', context)
    print('-----------------------------')

In [ ]:
trainer.test(datamodule=data_module)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/checkpoints/best-checkpoint.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/checkpoints/best-checkpoint.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    1.9897609949111938     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.9897609949111938}]

In [ ]:
sample_index = 1  # The valid indices for a DataFrame with 2 rows are 0 and 1
sample_question = test_df.iloc[sample_index]

generated = generate(best_model, sample_question['answer_text'], sample_question['context'])
show_result(generated, sample_question['answer_text'], sample_question['context'], sample_question['question'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Generated:  <pad> Mạng C2 hạt nhân <sep> Điều gì đã được sử dụng bởi các hoạt động của Fn?</s>
Original :  Những mạng quan trọng là nó chức năng của Cyberspace hỗ trợ để bảo vệ?

Answer:  Mạng C2 / liên kết truyền thông và mạng C2 hạt nhân
Conext:  Hỗ trợ không gian mạng là các hoạt động nền tảng, liên tục hoặc đáp ứng đảm bảo tính toàn vẹn và sẵn có của thông tin trong, thông qua và từ cơ sở hạ tầng do Không quân kiểm soát và phần tương tự và kỹ thuật số được kết nối với nhau của không gian chiến đấu. Kế thừa trong nhiệm vụ này là khả năng thiết lập, mở rộng, bảo mật, bảo vệ và bảo vệ để duy trì các mạng và nhiệm vụ được giao. Điều này bao gồm các biện pháp bảo vệ chống lại các thành phần chuỗi cung ứng cộng với các mạng / liên kết truyền thông quan trọng của C2 và mạng C2 hạt nhân. Nhiệm vụ hỗ trợ không gian mạng kết hợp các kỹ thuật CNE và CND. Nó kết hợp tất cả các yếu tố của Hoạt động mạng Không quân, vận chuyển thông tin, quản lý doanh nghiệp và đảm bảo thông tin, và phụ thuộc và

In [ ]:
# for i in range(min(len(test_df), 10)):
#     context = test_df.iloc[i]['context']
#     original_answer = test_df.iloc[i]['answer_text']
#     question = test_df.iloc[i]['question']

#     generated = generate(best_model, original_answer, context)

#     print(f"Question: {question}")
#     print(f"Original Answer: {original_answer}")
#     print(f"Generated Answer: {generated}")
#     print(f"Context: {context}")
#     print("="*50)


In [ ]:
def generate(qgmodel: QGModel, answer: str, context: str) -> str:
    source_encoding = tokenizer(
        '{} {} {}'.format(answer, SEP_TOKEN, context),
        max_length=SOURCE_MAX_TOKEN_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )


    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    best_model.to(device)

    input_ids = source_encoding['input_ids'].to(device)
    attention_mask = source_encoding['attention_mask'].to(device)

    generated_ids = qgmodel.model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        num_beams=1,
        max_length=TARGET_MAX_TOKEN_LEN,
        repetition_penalty=1.0,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=True
    )


    generated_question = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return generated_question

def show_result(generated: str, answer: str, context:str, original_question: str = ''):
    print('Generated Question: ', generated)
    if original_question:
        print('Original Question: ', original_question)

    print()
    print('Answer: ', answer)
    print('Context: ', context)
    print('-----------------------------')

sample_question = test_df.iloc[42]

generated_question = generate(best_model, sample_question['answer_text'], sample_question['context'])
show_result(generated_question, sample_question['answer_text'], sample_question['context'], sample_question['question'])


Generated Question:  đồng hồ đo ánh sáng sinh học <sep> Điều gì đã được tạo ra để đo lường ánh sáng sinh học?
Original Question:  Một Daysimeter loại đồng hồ cá nhân?

Answer:  đồng hồ đo ánh sáng sinh học
Context:  Để đo cụ thể lượng ánh sáng đi vào mắt, đồng hồ đo ánh sáng sinh học cá nhân được gọi là Daysimeter đã được phát triển. Đây là thiết bị đầu tiên được tạo ra để đo lường và mô tả chính xác ánh sáng (cường độ, phổ, thời gian và thời lượng) đi vào mắt ảnh hưởng đến đồng hồ của cơ thể con người.
-----------------------------


In [ ]:
def generate_question_and_answer(qgmodel: QGModel, context: str):

    source_encoding = tokenizer(
        '{} {}'.format(context, SEP_TOKEN),
        max_length=SOURCE_MAX_TOKEN_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    qgmodel.to(device)

    input_ids = source_encoding['input_ids'].to(device)
    attention_mask = source_encoding['attention_mask'].to(device)

    generated_ids = qgmodel.model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        num_beams=1,
        max_length=TARGET_MAX_TOKEN_LEN,
        repetition_penalty=1.0,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=True
    )

    generated_question = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return generated_question, context




#Test
Cấu trúc \<answer> + \<sep> + \<question>

## Test Case 1

In [ ]:
context = "Con người là một sinh vật sống bậc cao, nhưng con gà thì không "
generated_question, generated_answer = generate_question_and_answer(best_model, context)

print("Generated Question:", generated_question)
print("Context:", generated_answer)

print()
context = \
"""
Vào tháng 1 năm 1902, với số nợ 2.670 bảng – tương đương với 310.000 bảng năm 2024[nb 1] – đội đứng trước nguy cơ bị giải thể.[18] Đội trưởng Harry
 Stafford sau đó gặp được bốn thương nhân địa phương, trong đó có John Henry Davies (người trở thành chủ tịch câu lạc bộ), mỗi người sẵn lòng bỏ ra 500
 bảng để đổi lấy quyền điều hành câu lạc bộ và cũng là những người đổi tên câu lạc bộ.[19] Ngày 24 tháng 4 năm 1902, Manchester United chính thức ra đời.
  Dưới sự dẫn dắt của Ernest Mangnall, người được bổ nhiệm vào năm 1903, đội giành vị trí á quân Giải hạng nhì năm 1906 và giành quyền lên chơi ở Giải hạng nhất.
  Vào năm 1908 đội giành chức vô địch quốc gia đầu tiên. Mùa giải sau đó United khởi đầu với chức vô địch Charity Shield[21] và
 kết thúc với chức vô địch Cúp FA. Manchester United vô địch Hạng nhất lần thứ hai vào năm 1911, nhưng cuối mùa giải sau đó, Mangnall chia tay đội để chuyển đến Manchester City.
"""
generated_question, generated_answer = generate_question_and_answer(best_model, context)

print("Generated Question:", generated_question)
print("Context:", generated_answer)


Generated Question: Con người là một sinh vật sống bậc cao, nhưng con gà, không <sep> Những con người nào đã được sinh vật sống bậc cao?
Context: Con người là một sinh vật sống bậc cao, nhưng con gà thì không 

Generated Question: Tháng 1 năm 1911, và cúp FA. <sep> Đội trưởng William Stafford đã giành chiến thắng trong năm nào?
Context: 
Vào tháng 1 năm 1902, với số nợ 2.670 bảng – tương đương với 310.000 bảng năm 2024[nb 1] – đội đứng trước nguy cơ bị giải thể.[18] Đội trưởng Harry
 Stafford sau đó gặp được bốn thương nhân địa phương, trong đó có John Henry Davies (người trở thành chủ tịch câu lạc bộ), mỗi người sẵn lòng bỏ ra 500 
 bảng để đổi lấy quyền điều hành câu lạc bộ và cũng là những người đổi tên câu lạc bộ.[19] Ngày 24 tháng 4 năm 1902, Manchester United chính thức ra đời.
  Dưới sự dẫn dắt của Ernest Mangnall, người được bổ nhiệm vào năm 1903, đội giành vị trí á quân Giải hạng nhì năm 1906 và giành quyền lên chơi ở Giải hạng nhất. 
  Vào năm 1908 đội giành chức vô địch quốc

## Test case 2

In [ ]:
context = \
"""
Khi thay nhóm OH của nhóm cacboxyl trong phân tử axit cacboxylic bằng nhóm OR ta được hợp chất este.
Đặc điểm cấu tạo: Trong phân tử este của axit cacboxylic có nhóm COOR, với R là gốc hiđrocacbon.
Este no, đơn chức, mạch hở có công thức phân tử CnH2nO2 n lớn hoặc bằng hơn 2.
"""
generated_question, generated_answer = generate_question_and_answer(best_model, context)

print("Generated Question:", generated_question)
print("Context:", generated_answer)


Generated Question: R là gốc hiđrocacbon. <sep> Những gì có công thức phân tử axit cacboxylic bằng nhóm?
Context: 
Khi thay nhóm OH của nhóm cacboxyl trong phân tử axit cacboxylic bằng nhóm OR ta được hợp chất este.
Đặc điểm cấu tạo: Trong phân tử este của axit cacboxylic có nhóm COOR, với R là gốc hiđrocacbon.
Este no, đơn chức, mạch hở có công thức phân tử CnH2nO2 n lớn hoặc bằng hơn 2.



## Test case 3

In [ ]:
context = \
"""
Cho tập hợp A gồm n phần tử với n lớn hơn hoặc bằng 1.Mỗi kết quả của sự sắp xếp thứ tự n phần tử của tập
hợp A được gọi là một hoán vị của n phần tử đó
"""
generated_question, generated_answer = generate_question_and_answer(best_model, context)

print("Generated Question:", generated_question)
print("Context:", generated_answer)


Generated Question: 1.Mỗi kết quả của sự sắp xếp thứ tự n phần tử của tập hợp A được gọi là một hoán vị của n phần tử với n phần tử với n phần tử với n phần tử với n phần tử với n phần tử với n phần tử với n phần tử với n phần tử với n phần tử với n phần tử với n phần tử với n phần tử với n
Context: 
Cho tập hợp A gồm n phần tử với n lớn hơn hoặc bằng 1.Mỗi kết quả của sự sắp xếp thứ tự n phần tử của tập
hợp A được gọi là một hoán vị của n phần tử đó



In [ ]:
context = \
"""
Định lý này đã làm khó không biết bao bộ óc vĩ đại của các nhà toán học lừng danh trong gần 4 thế kỉ.
Cuối cùng nó được Andrew Wiles chứng minh vào năm 1993 sau gần 8 năm ròng nghiên cứu,
phát triển từ chứng minh các giả thiết có liên quan. Tuy nhiên chứng minh này còn thiếu sót và đến năm 1995 Wiles mới hoàn tất,
công bố chứng minh trọn vẹn sau 358 năm nỗ lực chứng minh của các nhà toán học.
Bằng chứng được mô tả là một 'bước tiến tuyệt vời' trong trích dẫn cho giải thưởng Abel năm 2016.
Bằng chứng của Định lý cuối cùng của Fermat cũng đã chứng minh được rất nhiều định lý module và
mở ra toàn bộ các phương pháp tiếp cận mới cho nhiều vấn đề khác và nâng tầm kỹ thuật tính toán module.
Những vấn đề chưa được giải quyết đã thúc đẩy sự phát triển của lý thuyết đại số ở thế kỉ 19 và sự chứng minh của định lý Module ở thế kỉ 20.
Đây là định lý trứ danh nhất trong lịch sử toán học.
Trước khi nó được chứng minh thì định lý đã được ghi vào sách kỷ lục Guinness
thế giới như là một vấn đề toán học khó nhất mọi thời đại, một trong những lý do định lý này được gọi như vậy là vì có một lượng khổng lồ các bài chứng minh không thành công.
"""
generated_question, generated_answer = generate_question_and_answer(best_model, context)

print("Generated Question:", generated_question)
print("Context:", generated_answer)


Generated Question: 19 và sự chứng minh của các bài chứng minh không thành công. <sep> Điều gì đã được phát triển bởi sự phát triển của lý thuyết của Fermat?
Context: 
Định lý này đã làm khó không biết bao bộ óc vĩ đại của các nhà toán học lừng danh trong gần 4 thế kỉ. 
Cuối cùng nó được Andrew Wiles chứng minh vào năm 1993 sau gần 8 năm ròng nghiên cứu, 
phát triển từ chứng minh các giả thiết có liên quan. Tuy nhiên chứng minh này còn thiếu sót và đến năm 1995 Wiles mới hoàn tất, 
công bố chứng minh trọn vẹn sau 358 năm nỗ lực chứng minh của các nhà toán học. 
Bằng chứng được mô tả là một 'bước tiến tuyệt vời' trong trích dẫn cho giải thưởng Abel năm 2016. 
Bằng chứng của Định lý cuối cùng của Fermat cũng đã chứng minh được rất nhiều định lý module và 
mở ra toàn bộ các phương pháp tiếp cận mới cho nhiều vấn đề khác và nâng tầm kỹ thuật tính toán module. 
Những vấn đề chưa được giải quyết đã thúc đẩy sự phát triển của lý thuyết đại số ở thế kỉ 19 và sự chứng minh của định lý Module ở 

# Đánh giá:
- Đào tạo chưa đủ nhiều (tránh mất quá nhiều thời gian), dữ liệu chưa đủ, training lâu
- Ưu: phán đoán tốt với text có nghĩa rõ ràng (gần với tập train)
- Yếu khi đưa ra đánh giá với đoạn context hàn lâm, khoa học (dữ liệu chưa đủ bao quát)

In [ ]:
from google.colab import files
import shutil

# Compress the file
shutil.make_archive('/content/best-checkpoint', 'zip', '/content/checkpoints', 'best-checkpoint.ckpt')

# Download the compressed file
files.download('/content/best-checkpoint.zip')
